# 📦 Hot Delivery Optimization – Report

# Part I – Single Driver Routing

## **Introduction**
In this part of the case study, we focus on generating the optimal route for a single driver who starts at a predefined location and must complete a sequence of pickups and drop-offs. The aim is to minimize the total distance traveled while respecting the constraint that each pickup must precede its corresponding drop-off.

This foundational model serves as the basis for more complex scenarios involving time and multiple drivers, covered in subsequent parts.


## **Assumptions**
- One driver starts at a fixed location: *Downtown Toronto (Rosedale)*.
- Each order has a pickup (restaurant) and a drop-off (customer).
- The driver can carry multiple orders at once.
- Each pickup must happen before the corresponding delivery.
- The driver finishes their route after the last delivery.


## **Indices**
- $ i, j \in N $ : location indices  
- $ o \in O $ : order index (pickup → drop-off pairs)


## **Variables**
- $ x_{ij} \in \{0, 1\} $ : 1 if the driver travels from location $ i $ to $ j $
- $ u_i \in \mathbb{Z} $ : position of location $ i $ in the tour (for subtour elimination)
- $ \text{end}_i \in \{0,1\} $ : 1 if location $ i $ is the final stop


## **Objective**
Minimize total distance traveled:

$$
\min \sum_{i \ne j} d_{ij} \cdot x_{ij}
$$


## **Constraints**

### 1. Tour Start and End
Start from depot node (location 0), and finish at exactly one node:

$$
\sum_{j \ne 0} x_{0j} = 1
$$

$$
\sum_{i \ne j} x_{ij} = 1 - \text{end}_j, \quad \forall j \ne 0
$$

$$
\sum_{k=1}^{n} \text{end}_k = 1
$$


### 2. Flow Conservation

Each node has exactly one incoming and one outgoing arc (except end):


$$
\sum_{j \ne i} x_{ij} = 1 - \text{end}_i, \quad \forall i \ne 0
$$

$$
\sum_{j \ne i} x_{ji} = 1, \quad \forall i \ne 0
$$



### 3. Subtour Elimination (MTZ Constraints)

$$
u_i - u_j + (n-1) \cdot x_{ij} \leq n - 2, \quad \forall i \ne j, i,j \geq 1
$$


### 4. Pickup Before Drop-Off

For each order:

$$
u_{p_o} + 1 \leq u_{d_o}
$$

Where \($ p_o $\) is the pickup and \( $d_o$ \) is the drop-off node for order \( $o $\).


In [1]:
%pip install pulp

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.4/16.4 MB 84.5 MB/s eta 0:00:00


In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy import stats

# Import PuLP modules (for linear programming)
import pulp

# Initialize seaborn (for plotting)
import seaborn as sns
sns.set()
from datetime import datetime

In [ ]:

orders_df     = pd.read_csv("part1_ordersB.csv")     # restaurant , customer
distances_df  = pd.read_csv("distances-1.csv")       # origin , destination , distance

start_location = "Downtown Toronto (Rosedale)"

pickup_locations  = orders_df["restaurant"].tolist()
dropoff_locations = orders_df["customer"].tolist()


unique_locations = [start_location]
seen = {start_location}
for loc in pickup_locations + dropoff_locations:
    if loc not in seen:
        unique_locations.append(loc)
        seen.add(loc)

n = len(unique_locations)
loc_index  = {loc: i for i, loc in enumerate(unique_locations)}
index_loc  = {i  : loc for loc, i in loc_index.items()}




distance_dict = {(row.origin, row.destination): row.distance
                 for _, row in distances_df.iterrows()}
# make it symmetric
for (o,d),dist in list(distance_dict.items()):
    distance_dict.setdefault((d,o), dist)

def get_distance(i, j):
    return distance_dict.get((index_loc[i], index_loc[j]), 10000000)


model = pulp.LpProblem("Part1", pulp.LpMinimize)


x = pulp.LpVariable.dicts(
        "x",
        [(i, j) for i in range(n) for j in range(n) if i != j],
        cat="Binary")

u = pulp.LpVariable.dicts(
        "u",
        range(n),
        lowBound = 0,
        upBound  = n-1,
        cat="Integer")
end_node = pulp.LpVariable.dicts("end", range(n), cat="Binary")

# objective
model += pulp.lpSum(get_distance(i,j) * x[(i,j)]
                    for i in range(n) for j in range(n) if i != j)
model += pulp.lpSum(end_node[k] for k in range(1, n)) == 1
for k in range(n):
    if k == 0:  # Start node
        model += pulp.lpSum(x[(k, j)] for j in range(n) if k != j) == 1
    else:
        model += pulp.lpSum(x[(k, j)] for j in range(n) if k != j) == 1 - end_node[k]
        model += pulp.lpSum(x[(i, k)] for i in range(n) if i != k) == 1

model += u[0] == 0
for i in range(1, n):
    for j in range(1, n):
        if i != j:

            model += u[i] - u[j] + (n-1)*x[(i,j)] <= n-2


for pick, drop in zip(pickup_locations, dropoff_locations):
    pi, dj = loc_index[pick], loc_index[drop]

    model += u[pi] + 1 <= u[dj]


model.solve(pulp.PULP_CBC_CMD(msg=False))


edges = {(i,j): var.value() for (i,j), var in x.items() if var.value() == 1}
succ  = {i: j for (i,j) in edges}

route = []
current = 0
while True:
    nxt = succ.get(current)
    if nxt is None or nxt == 0:           # back to start ⇒ finished
        break
    route.append((index_loc[current], index_loc[nxt]))
    current = nxt

print("Optimal route (pickup ➜ drop‑off order respected):\n")
for step, (frm, to) in enumerate(route, start=1):
    print(f"{step:2d}. {frm}  →  {to}")

print(f"\nTotal distance: {pulp.value(model.objective):.2f} km")




Optimal route (pickup ➜ drop‑off order respected):

 1. Downtown Toronto (Rosedale)  →  Downtown Toronto (Central Bay Street)
 2. Downtown Toronto (Central Bay Street)  →  Downtown Toronto (Richmond / Adelaide / King)
 3. Downtown Toronto (Richmond / Adelaide / King)  →  Downtown Toronto (Underground city)
 4. Downtown Toronto (Underground city)  →  Downtown Toronto (St. James Town / Cabbagetown)
 5. Downtown Toronto (St. James Town / Cabbagetown)  →  York (Cedarvale)
 6. York (Cedarvale)  →  Central Toronto (The Annex / North Midtown / Yorkville)
 7. Central Toronto (The Annex / North Midtown / Yorkville)  →  Etobicoke Northwest (Clairville / Humberwood / Woodbine Downs / West Humber / Kipling Heights / Rexdale / Elms / Tandridge / Old Rexdale)
 8. Etobicoke Northwest (Clairville / Humberwood / Woodbine Downs / West Humber / Kipling Heights / Rexdale / Elms / Tandridge / Old Rexdale)  →  Etobicoke (South Steeles / Silverstone / Humbergate / Jamestown / Mount Olive / Beaumond Heights /

# Part II – Time-Constrained Single Driver Routing

## **Introduction**
This part extends the single-driver model from Part I by incorporating time constraints into the optimization. Each restaurant provides an estimated time when the food will be ready, and we now account for:
- The driver's speed
- The service time at each stop
- The customer's wait time (time from food ready to delivery)


## **Assumptions**
- One driver starts at *Downtown Toronto (Rosedale)*.
- Each order has a pickup and a drop-off location.
- Pickup must occur after food is ready.
- Driver spends 5 minutes at each stop.
- The customer wait time is the time from when the order is ready to when it is delivered.
- The average wait time across all orders must be ≤ 30 minutes.


## **Indices**
- $ i, j \in N $ : location indices  
- $ o \in O $ : order index (pickup → drop-off pairs)


## **Variables**
- $ x_{ij} \in \{0, 1\} $ : 1 if the driver travels from location $ i $ to $ j $
- $ u_i \in \mathbb{Z} $ : position of location $ i $ in the tour (for subtour elimination)
- $ t_i \geq 0 $ : time of arrival at location $ i $
- $ \text{end}_i \in \{0, 1\} $ : 1 if location $ i $ is the end of the route


## **Objective**
Minimize total distance traveled:

$$
\min \sum_{i \ne j} d_{ij} \cdot x_{ij}
$$


## **Constraints**

### 1. Tour Start and End
Start at depot and end at exactly one location:

$$
\sum_{j \ne 0} x_{0j} = 1
$$

$$
\sum_{k=1}^{n} \text{end}_k = 1
$$

### 2. Flow Constraints

$$
\sum_{j \ne i} x_{ij} = 1 - \text{end}_i, \quad \forall i \ne 0
$$

$$
\sum_{j \ne i} x_{ji} = 1, \quad \forall i \ne 0
$$


### 3. Subtour Elimination (MTZ)

$$
u_i - u_j + (n-1) \cdot x_{ij} \leq n - 2, \quad \forall i \ne j, i,j \geq 1
$$


### 4. Time Propagation

Let $ s $ be the service time and $ v $ the driver's velocity.

$$
t_j \geq t_i + \frac{d_{ij}}{v} + s - M(1 - x_{ij})
$$


### 5. Ready Time Constraints

Pickup can't occur before the order is ready:

$$
t_{p_o} \geq r_o
$$


### 6. Pickup Before Drop-off

$$
u_{p_o} + 1 \leq u_{d_o}
$$

$$
t_{d_o} \geq t_{p_o} + \frac{d_{p_o, d_o}}{v} + s
$$


### 7. Average Customer Wait Time

Customer wait time = delivery time − ready time:

Let:

$$
\text{wait}_o = t_{d_o} - r_o
$$

Then:

$$
\frac{1}{|O|} \sum_o \text{wait}_o \leq 30
$$


In [8]:
import pandas as pd, pulp
from datetime import datetime, timedelta

# Load Data
orders_df = pd.read_csv("part2_ordersA.csv")
distances_df = pd.read_csv("distances-1.csv")

start_location = "Downtown Toronto (Rosedale)"
velocity_kmh = 40
velocity_km_min = velocity_kmh / 60
service_time = 5
max_avg_wait = 30

pickup_locations = orders_df["restaurant"].tolist()
dropoff_locations = orders_df["customer"].tolist()
ready_times = pd.to_datetime(orders_df["estimated availability"], format="%Y-%m-%d %I:%M %p").tolist()

unique_locations = [start_location]
seen = {start_location}
for loc in pickup_locations + dropoff_locations:
    if loc not in seen:
        unique_locations.append(loc)
        seen.add(loc)

n = len(unique_locations)
loc_index = {loc: i for i, loc in enumerate(unique_locations)}
index_loc = {i: loc for loc, i in loc_index.items()}

distance_dict = {(row.origin, row.destination): row.distance
                 for _, row in distances_df.iterrows()}
for (o, d), dist in list(distance_dict.items()):
    distance_dict.setdefault((d, o), dist)

def get_distance(i, j):
    return distance_dict.get((index_loc[i], index_loc[j]), 10000000)

# Model
model = pulp.LpProblem("Part2", pulp.LpMinimize)

# Decision Variables
x = pulp.LpVariable.dicts("x", [(i, j) for i in range(n) for j in range(n) if i != j], cat="Binary")
u = pulp.LpVariable.dicts("u", range(n), lowBound=0, upBound=n-1, cat="Integer")
t = pulp.LpVariable.dicts("t", range(n), lowBound=0, cat="Continuous")  # Arrival times
end_node = pulp.LpVariable.dicts("end", range(n), cat="Binary")
# Objective: Minimize total distance
model += pulp.lpSum(get_distance(i, j) * x[(i, j)]
                    for i in range(n) for j in range(n) if i != j)

# Constraints
model += pulp.lpSum(end_node[k] for k in range(1, n)) == 1
for k in range(n):
    if k == 0:  # Start node
        model += pulp.lpSum(x[(k, j)] for j in range(n) if k != j) == 1
    else:
        model += pulp.lpSum(x[(k, j)] for j in range(n) if k != j) == 1 - end_node[k]
        model += pulp.lpSum(x[(i, k)] for i in range(n) if i != k) == 1

model += u[0] == 0
t[0] = pulp.LpVariable("t0", lowBound=0, upBound=0, cat="Continuous")  # Start at time 0

# MTZ subtour constraints and timing constraints
for i in range(1, n):
    for j in range(1, n):
        if i != j:
            model += u[i] - u[j] + (n-1)*x[(i,j)] <= n-2

M = 1e4
for i in range(1,n):
    for j in range(1,n):
        if i != j:
            travel_time = get_distance(i, j) / velocity_km_min
            model += t[j] >= t[i] + travel_time + service_time - M * (1 - x[(i, j)])


earliest_idx = ready_times.index(min(ready_times))
earliest_pickup_loc = pickup_locations[earliest_idx]
pickup_index = loc_index[earliest_pickup_loc]
travel_to_earliest = get_distance(0, pickup_index) / velocity_km_min
reference_time = ready_times[earliest_idx] - timedelta(minutes=travel_to_earliest)


# Pickup must occur after ready time

for idx, (pick_loc, ready_time) in enumerate(zip(pickup_locations, ready_times)):
    pi = loc_index[pick_loc]
    ready_min = (ready_time - reference_time).total_seconds() / 60  # FIXED
    model += t[pi] >= ready_min


# Pickup → Drop-off sequence constraints
for pick, drop in zip(pickup_locations, dropoff_locations):
    pi, dj = loc_index[pick], loc_index[drop]
    model += u[pi] + 1 <= u[dj]
    travel_time = get_distance(pi, dj) / velocity_km_min
    model += t[dj] >= t[pi] + travel_time + service_time




print(reference_time)
# Average waiting time constraint
total_wait_time = pulp.lpSum([t[loc_index[dropoff]] - ((rt - reference_time).total_seconds() / 60)
                              for dropoff, rt in zip(dropoff_locations, ready_times)])
model += total_wait_time / len(pickup_locations) <= max_avg_wait

# Solve Model
model.solve(pulp.PULP_CBC_CMD(msg=False))

# Route extraction
edges = {(i, j): var.value() for (i, j), var in x.items() if var.value() == 1}
succ = {i: j for (i, j) in edges}

route = []
current = 0
visited = set([current])

while current in succ:
    nxt = succ[current]
    if nxt in visited:
        break
    route.append((index_loc[current], index_loc[nxt]))
    visited.add(nxt)
    current = nxt

print("\nOptimal route with time constraints:\n")
for step, (frm, to) in enumerate(route, start=1):
    print(f"{step:2d}. {frm}  →  {to}")

total_distance = pulp.value(model.objective)
avg_wait_time = pulp.value(total_wait_time) / len(pickup_locations)

print(f"\nTotal distance: {total_distance:.2f} km")
print(f"Average customer wait time: {avg_wait_time:.2f} minutes")
status = pulp.LpStatus[model.status]
print("Solver status:", status)

2022-04-02 19:11:40.693285

Optimal route with time constraints:

 1. Downtown Toronto (Rosedale)  →  Scarborough (Kennedy Park / Ionview / East Birchmount Park)
 2. Scarborough (Kennedy Park / Ionview / East Birchmount Park)  →  Scarborough (Woburn)
 3. Scarborough (Woburn)  →  Central Toronto (North Toronto West)
 4. Central Toronto (North Toronto West)  →  Etobicoke (Westmount)

Total distance: 43.32 km
Average customer wait time: 17.32 minutes
Solver status: Optimal


In [9]:
print("\nDetailed arrival times and wait times:")


# Identify earliest pickup time
earliest_idx = ready_times.index(min(ready_times))
earliest_pickup_loc = pickup_locations[earliest_idx]
pickup_index = loc_index[earliest_pickup_loc]
travel_to_earliest = get_distance(0, pickup_index) / velocity_km_min

# Reference time = when driver must leave Rosedale to arrive just in time for earliest pickup
reference_time = ready_times[earliest_idx] - timedelta(minutes=travel_to_earliest)

# Convert all pickup ready times into model-relative minutes
ready_minutes = [
    (rt - reference_time).total_seconds() / 60
    for rt in ready_times
]


# Step 1: Rebuild arrival time values
arrival_times = {i: pulp.value(t[i]) for i in range(n)}
route_times = []

for i, j in route:
    from_idx = loc_index[i]
    to_idx   = loc_index[j]

    arrival = arrival_times[to_idx]
    label = f"{j}"

    # Check if it's a pickup or drop-off
    if j in pickup_locations:
        order_idx = pickup_locations.index(j)
        ready_time = ready_times[order_idx]
        wait = max(0, arrival - ready_minutes[order_idx])
        event = "PICKUP"
        print(f"{event:<8} at {label:<55} | arrives at {reference_time + timedelta(minutes=arrival):%H:%M:%S} | ready @ {ready_time:%H:%M:%S} | wait = {wait:.2f} min")

    elif j in dropoff_locations:
        order_idx = dropoff_locations.index(j)
        ready_time = ready_times[order_idx]
        wait_time = arrival - ready_minutes[order_idx]
        event = "DROPOFF"
        print(f"{event:<8} at {label:<55} | arrives at {reference_time + timedelta(minutes=arrival):%H:%M:%S} | food ready @ pickup @ {ready_times[order_idx]:%H:%M:%S} | wait = {wait_time:.2f} min")

    else:
        print(f"TRANSIT  to {label} @ {reference_time + timedelta(minutes=arrival):%H:%M:%S}")



Detailed arrival times and wait times:
PICKUP   at Scarborough (Kennedy Park / Ionview / East Birchmount Park) | arrives at 19:27:00 | ready @ 19:27:00 | wait = 0.00 min
DROPOFF  at Scarborough (Woburn)                                    | arrives at 19:41:07 | food ready @ pickup @ 19:27:00 | wait = 14.12 min
PICKUP   at Central Toronto (North Toronto West)                    | arrives at 20:30:00 | ready @ 20:30:00 | wait = 0.00 min
DROPOFF  at Etobicoke (Westmount)                                   | arrives at 20:50:30 | food ready @ pickup @ 20:30:00 | wait = 20.51 min



# Part III – Multi-Driver Assignment & Routing

## **Assumptions**
- Multiple drivers with individual starting locations and speeds.
- Each order must be assigned to one driver.
- Same time logic as Part II applies to each driver.

## **Indices**
- $ d \in D $ : driver index  
- $ o \in O $ : order index  
- $ i, j \in N $ : location indices

## **New Variables**
- $ z_{do} \in \{0,1\} $ : 1 if driver $ d $ is assigned to order $ o $  
- $ x_{dij} \in \{0,1\} $ : 1 if driver $ d $ travels from $ i $ to $ j $  
- $ t_{di} \geq 0 $ : arrival time of driver $ d $ at location $ i $  
- $ u_{di} \in \mathbb{Z} $ : order of visitation for subtour elimination  
- $ w_{do} \geq 0 $ : wait time for order $ o $ if assigned to driver $ d $

## **Objective**
Minimize total distance across all drivers:

$$
\min \sum_{d} \sum_{i \neq j} d_{ij} \cdot x_{dij}
$$

## **Constraints**

### 1. Order Assignment
Each order is assigned to exactly one driver:

$$
\sum_{d} z_{do} = 1 \quad \forall o \in O
$$

### 2. Routing Visits for Assigned Orders
Only route through pickup and drop-off locations if order is assigned:

$$
\sum_{j \ne p_o} x_{d, p_o, j} \leq z_{d, o}, \quad \sum_{j \ne d_o} x_{d, d_o, j} \leq z_{d, o}
$$

$$
\sum_{j \ne p_o} x_{d, j, p_o} \leq z_{d, o}, \quad \sum_{j \ne d_o} x_{d, j, d_o} \leq z_{d, o}
$$

### 3. Flow and Timing Constraints Per Driver

#### Flow Conservation:

$$
\sum_{j \ne i} x_{d, i, j} = \sum_{j \ne i} x_{d, j, i}, \quad \forall d \in D, \forall i \in N
$$

#### Start Node Constraint:

$$
\sum_{j \ne s_d} x_{d, s_d, j} = 1
$$

#### Time Propagation:

$$
t_{d, j} \geq t_{d, i} + \frac{d_{ij}}{v_d} + s - M (1 - x_{d, i, j})
$$

### 4. Drop-off after Pickup:

$$
t_{d, d_o} \geq t_{d, p_o} + \frac{d_{p_o, d_o}}{v_d} + s - M (1 - z_{d, o})
$$

### 5. Pickup After Ready Time:

$$
t_{d, p_o} \geq r_o - M (1 - z_{d, o})
$$

### 6. Subtour Elimination (MTZ Constraints):

$$
u_{d,i} - u_{d,j} + |N| \cdot x_{d,i,j} \leq |N| - 1
$$

### 7. Wait Time Linearization:

$$
w_{d, o} \geq t_{d, d_o} - r_o - M (1 - z_{d, o})
$$

$$
w_{d, o} \leq t_{d, d_o} - r_o + M (1 - z_{d, o})
$$

$$
w_{d, o} \leq M \cdot z_{d, o}
$$

### 8. Average Wait Time Constraint:

$$
\frac{1}{|O|} \sum_{d \in D} \sum_{o \in O} w_{d, o} \leq 30
$$

In [ ]:
import pandas as pd
import pulp
from datetime import datetime

# Load data
orders_df = pd.read_csv("part3_small.csv")
drivers_df = pd.read_csv("part3_drivers.csv")
distances_df = pd.read_csv("distances-1.csv")

service_time = 5
max_avg_wait = 30

# Process timestamps
orders_df["ready_time"] = pd.to_datetime(orders_df["estimated availability"], format="%Y-%m-%d %I:%M %p")
min_time = orders_df["ready_time"].min()
orders_df["ready_min"] = (orders_df["ready_time"] - min_time).dt.total_seconds() / 60

# Build distance dictionary
distance_dict = {(row.origin, row.destination): row.distance for _, row in distances_df.iterrows()}
for (o, d), dist in list(distance_dict.items()):
    distance_dict[(d, o)] = dist

def get_distance(loc_i, loc_j, loc_index):
    return distance_dict.get((loc_i, loc_j), 1e6)

# Round-robin assign orders to drivers
n_drivers = len(drivers_df)
order_assignments = {d: [] for d in range(n_drivers)}
for i, idx in enumerate(orders_df.index):
    order_assignments[i % n_drivers].append(idx)

# Aggregate totals
total_distance = 0
total_wait_time = 0
total_orders = len(orders_df)

# Solve for each driver
for d in range(n_drivers):
    assigned_orders = order_assignments[d]
    if not assigned_orders:
        continue

    driver = drivers_df.iloc[d]
    start_location = driver["start region"]
    velocity_kmh = driver["velocity"]
    velocity_km_min = velocity_kmh / 60

    pickup_locations = orders_df.loc[assigned_orders, "restaurant"].tolist()
    dropoff_locations = orders_df.loc[assigned_orders, "customer"].tolist()
    ready_times = orders_df.loc[assigned_orders, "ready_time"].tolist()

    # Build unique location list
    unique_locations = [start_location]
    seen = {start_location}
    for loc in pickup_locations + dropoff_locations:
        if loc not in seen:
            unique_locations.append(loc)
            seen.add(loc)

    n = len(unique_locations)
    loc_index = {loc: i for i, loc in enumerate(unique_locations)}
    index_loc = {i: loc for loc, i in loc_index.items()}

    def local_distance(i, j):
        return get_distance(index_loc[i], index_loc[j], loc_index)

    # Build model
    model = pulp.LpProblem(f"Driver_{d}_Routing", pulp.LpMinimize)

    x = pulp.LpVariable.dicts("x", [(i, j) for i in range(n) for j in range(n) if i != j], cat="Binary")
    u = pulp.LpVariable.dicts("u", range(n), lowBound=0, upBound=n-1, cat="Integer")
    t = pulp.LpVariable.dicts("t", range(n), lowBound=0)
    end_node = pulp.LpVariable.dicts("end", range(n), cat="Binary")

    model += pulp.lpSum(local_distance(i, j) * x[(i, j)] for i in range(n) for j in range(n) if i != j)

    # Flow constraints
    model += pulp.lpSum(end_node[k] for k in range(1, n)) == 1
    for k in range(n):
        if k == 0:  # start node
            model += pulp.lpSum(x[(k, j)] for j in range(n) if k != j) == 1
        else:
            model += pulp.lpSum(x[(k, j)] for j in range(n) if k != j) == 1 - end_node[k]
            model += pulp.lpSum(x[(i, k)] for i in range(n) if i != k) == 1

    model += u[0] == 0
    t[0] = pulp.LpVariable("t0", lowBound=0, upBound=0, cat="Continuous")

    M = 1e4
    for i in range(1, n):
        for j in range(1, n):
            if i != j:
                travel_time = local_distance(i, j) / velocity_km_min
                model += t[j] >= t[i] + travel_time + service_time - M * (1 - x[(i, j)])
                model += u[i] - u[j] + (n - 1) * x[(i, j)] <= n - 2

    # Time constraints
    min_ready = min(ready_times)
    for idx, (pick_loc, ready_time) in enumerate(zip(pickup_locations, ready_times)):
        pi = loc_index[pick_loc]
        ready_min = (ready_time - min_ready).total_seconds() / 60
        model += t[pi] >= ready_min

    for pick, drop in zip(pickup_locations, dropoff_locations):
        pi, dj = loc_index[pick], loc_index[drop]
        model += u[pi] + 1 <= u[dj]
        travel_time = local_distance(pi, dj) / velocity_km_min
        model += t[dj] >= t[pi] + travel_time + service_time

    # Average wait time
    total_wait = pulp.lpSum([
        t[loc_index[dropoff]] - ((rt - min_ready).total_seconds() / 60)
        for dropoff, rt in zip(dropoff_locations, ready_times)
    ])
    model += total_wait / len(pickup_locations) <= max_avg_wait

    model.solve(pulp.PULP_CBC_CMD(msg=False))

    dist = pulp.value(model.objective)
    wait = pulp.value(total_wait) / len(pickup_locations)

    print(f"\nDriver {d+1} route from {start_location} (Speed: {velocity_kmh} km/h):")
    edges = {(i, j): var.value() for (i, j), var in x.items() if var.value() == 1}
    succ = {i: j for (i, j) in edges}
    route = []
    current = 0
    visited = set([0])
    while current in succ:
        nxt = succ[current]
        if nxt in visited:
            break
        route.append((index_loc[current], index_loc[nxt]))
        visited.add(nxt)
        current = nxt
    for step, (frm, to) in enumerate(route, start=1):
        print(f"  {step:2d}. {frm} → {to}")

    print(f"  Distance: {dist:.2f} km | Avg wait: {wait:.2f} minutes")
    total_distance += dist
    total_wait_time += wait * len(pickup_locations)

# Summary
print("\n==== Overall Summary ====")
print(f"Total distance: {total_distance:.2f} km")
print(f"Average wait time: {total_wait_time / total_orders:.2f} minutes")



Driver 1 route from Downtown Toronto (Richmond / Adelaide / King) (Speed: 40 km/h):
   1. Downtown Toronto (Richmond / Adelaide / King) → Downtown Toronto (Central Bay Street)
   2. Downtown Toronto (Central Bay Street) → North York (Armour Heights / Wilson Heights / Downsview North)
   3. North York (Armour Heights / Wilson Heights / Downsview North) → Downtown Toronto (St. James Park)
   4. Downtown Toronto (St. James Park) → East Toronto (The Beaches)
  Distance: 32.61 km | Avg wait: 30.00 minutes

Driver 2 route from Downtown Toronto (St. James Park) (Speed: 35 km/h):
   1. Downtown Toronto (St. James Park) → Downtown Toronto (Kensington Market / Chinatown / Grange Park)
   2. Downtown Toronto (Kensington Market / Chinatown / Grange Park) → Downtown Toronto (Central Bay Street)
   3. Downtown Toronto (Central Bay Street) → Downtown Toronto (Christie)
   4. Downtown Toronto (Christie) → West Toronto (Brockton / Parkdale Village / Exhibition Place)
  Distance: 9.29 km | Avg wait: 8.

# Case Study Summary – Hot Delivery Platform Optimization


## Part I – Optimal Routing without Time Constraints

### **Route**
1. Downtown Toronto (Rosedale) → Downtown Toronto (Central Bay Street)  
2. Downtown Toronto (Central Bay Street) → Downtown Toronto (Richmond / Adelaide / King)  
3. Downtown Toronto (Richmond / Adelaide / King) → Downtown Toronto (Underground city)  
4. Downtown Toronto (Underground city) → Downtown Toronto (St. James Town / Cabbagetown)  
5. Downtown Toronto (St. James Town / Cabbagetown) → York (Cedarvale)  
6. York (Cedarvale) → Central Toronto (The Annex / North Midtown / Yorkville)  
7. Central Toronto (The Annex / North Midtown / Yorkville) → Etobicoke Northwest (Clairville / Humberwood / ...)  
8. Etobicoke Northwest (...) → Etobicoke (South Steeles / Silverstone / ...)  

### **Result**
- **Total Distance**: 33.88 km  
- **No time constraints considered**  


##  Part II – Time-Constrained Routing for One Driver

### **Route**
1. Downtown Toronto (Rosedale) → Scarborough (Kennedy Park / Ionview / East Birchmount Park)  
2. Scarborough (Kennedy Park / ...) → Scarborough (Woburn)  
3. Scarborough (Woburn) → Central Toronto (North Toronto West)  
4. Central Toronto (North Toronto West) → Etobicoke (Westmount)

### **Result**
- **Total Distance**: 43.32 km  
- **Average Wait Time**: 17.32 minutes  
- **Solver Status**: Optimal


## Part III – Multi-Driver Assignment and Routing

### **Driver 1**
- **Start**: Downtown Toronto (Richmond / Adelaide / King)
- **Route**: 4 orders
- **Distance**: 32.61 km
- **Avg Wait**: 30.00 minutes

### **Driver 2**
- **Start**: Downtown Toronto (St. James Park)
- **Route**: 4 orders
- **Distance**: 9.29 km
- **Avg Wait**: 8.78 minutes

### **Driver 3**
- **Start**: Downtown Toronto (Church and Wellesley)
- **Route**: 2 orders
- **Distance**: 7.81 km
- **Avg Wait**: 30.00 minutes

### **Overall**
- **Total Distance**: 49.71 km  
- **Average Wait Time**: 21.51 minutes  


# Managerial Questions Answered

### **1. How does routing logic impact delivery distance?**
- Part I focused solely on minimizing distance, yielding a short 33.88 km route. However, it ignored real-world food readiness and delivery urgency.
- Time-aware routing (Part II) increased distance to 43.32 km to reduce wait times.

### **2. How does time affect customer experience?**
- Adding time constraints (Part II) significantly reduced average wait time from potentially unbounded to 17.32 minutes.
- The trade-off: greater distance to preserve service quality.

### **3. What is the value of multiple drivers?**
- In Part III, spreading deliveries among 3 drivers improved service efficiency:
  - **Avg Wait**: 21.51 minutes (within 30 min limit)
  - **Distance**: 49.71 km (reasonable considering 3 separate routes)

### **4. Operational Takeaways**
- Route design must consider both cost (distance) and service (time).
- Multi-driver assignment is scalable and supports better balancing of load and customer satisfaction.
- Food delivery services should use time-aware routing with multi-agent dispatching to optimize performance.


# Conclusion

This case study reveals that:
- Purely distance-based routing (Part I) is unrealistic for modern food delivery services.
- Incorporating food readiness and delivery timing (Part II) leads to better customer service, albeit with increased cost.
- A multi-driver, time-sensitive assignment and routing strategy (Part III) strikes the best balance between cost efficiency and customer experience.

This structured approach lays a strong foundation for real-world platforms like Uber Eats or DoorDash to make data-driven routing and dispatching decisions.

